<div align="right"><i>Peter Norvig<br>2012<br>updated 2019, 2023</i></div>

# Weighing Twelve Balls on a Scale: ①②③④ ⚖ ⑤⑥⑦⑧ 

Here is [a classic](https://en.wikipedia.org/wiki/Balance_puzzle) brain-teaser puzzle from 1945:  

- *You are given twelve identical-looking balls and a two-sided balance scale. One of the balls is of a different weight, although you don't know whether it is lighter or heavier. How can you use just three weighings of the scale to determine not only what the different ball is, but also whether it is lighter or heavier?*


I would like to solve not just this specific puzzle, but related puzzles that vary:
- The number of balls.
- The number of weighings.
- The possibilities for the odd ball: maybe we know that it some ball is lighter, and can not be heavier. Or maybe one possibility is that all the balls actually weigh the same.
- (However, it will never be the case that *two* or more balls are different from the rest. That would change the puzzle too much.)

If I'm going to solve a puzzle with dozens or hundreds of balls, I'd rather use a computer program, not pencil and paper (as was intended for the 1945 version of the puzzle). I originally wrote such a program in Lisp around 1980, ported it to Python in 2012, and decided to publish it here as a notebook after seeing continued interest in the puzzle:
- A [Numberplay column](https://wordplay.blogs.nytimes.com/2014/07/21/12coin/) in 2014.
- A [MathWorld article](https://mathworld.wolfram.com/Weighing.html) in 2017.
- A [Wikipedia article](https://en.wikipedia.org/wiki/Balance_puzzle) that was started in 2018.
- A [538 Riddler column](https://fivethirtyeight.com/features/which-billiard-ball-is-rigged/)  2019.
- A [Mind-Benders for the Quarantined! entry](https://momath.org/civicrm/?page=CiviCRM&q=civicrm/event/info&reset=1&id=1620) in 2020. 


# Defining the Vocabulary
    
Here are the main concepts, and how I implement them in Python:

- **Puzzle**: A dataclass holding a description of the number of balls, number of allowable weighings, and the possible oddballs.
- **Ball**: A positive integer from 1 to some *N*. 
- **Oddball**: The one ball that is different from the others, and the way it is different: heavier or lighter. I'll use, e.g., `+3` to mean that ball `3` is heavier than the rest, `-3` to mean that ball `3` is lighter than the rest, and `0` ito mean that all the balls actually weigh the same.
- **Weighing**:  a dataclass describing the set of balls on each side of the scale, and the three possible outcomes of the weighing.
- **Outcome**: The outcome of a weighing is that the weight of the left side is greater than, equal to, or less than the right side.
- **Partition**: Each weighing  *partitions* the set of possible oddballs into three subsets, corresponding to the three possible outcomes.
- **Rule of 3**: A key insight is that two weighings partition the set of possible oddballs into 3 x 3 = 9 subsets, three weighings into 3 x 3 x 3 = 27 subsets, and <i>w</i> weighings into  3<sup><i>w</i></sup> subsets.  
- **Good/Bad Partitions**: A weighing that leaves more  3<sup>(<i>w</i>-1)</sup> oddballs in any outcome branch is said to be a **bad partition**, and could never lead to a solution. A weighing that does not do that is a **good partition** (but might not lead to a solution).`
- **Policy tree**: A tree where the interior nodes are weighings and rthe leaves are sets of oddballs. 
- **Solution**: A solution to a puzzle is a policy tree in which no path has more than the allowable number of weighings; every possible path ends with the identification of a single oddball; and all the oddballs are covered.

# Implementation of Basic Data Types and Constants

Some imports and definitions of data types and constants:

In [1]:
from dataclasses import dataclass
from typing import *
import random
random.seed(42) # For reproducability

empty      = frozenset()           # The empty set
lt, eq, gt = 'lt', 'eq', 'gt'      # The 3 possible outcomes of a weighing (`lt` means the left weighs less than the right)
Outcome    = Literal[lt, eq, gt]   # Possible outcomes of a weighing
Ball       = int                   # Balls are positive integers
Oddball    = int                   # An Oddball is an integer: positive (heavier), negative (lighter), or zero (same weight)
Tree       = Union['Weighing', Set[Oddball]] # A policy tree has Weighings for interior nodes and oddball sets for leaf nodes

# The `Puzzle` class

`Puzzle` is a data class that records the number of balls, the number of allowable weighings, a collection of ball numbers, and the possible oddballs. 

In [94]:
@dataclass
class Puzzle:
    """A ball-weighing puzzle."""
    N:         int              # the number of balls that can be weighed
    weighings: int              # the number of weighins
    balls:     Collection[Ball] # all the balls in the puzzle
    oddballs:  Set[Oddball]     # the allowable oddball numbers


The convenience function `make_puzzle` allows you to specify the oddballs by setting the parameter `odd` to be either a set of oddball integers, or a string of characters:

In [95]:
def make_puzzle(N, weighings, odd='+-'):
    """Create a puzzle with `N` balls, and the given number of `weighings`.
    `odd` is a collection of numbers, or a string with some of the characters `+-=`,
    meaning any balls might be heavy (`+`), light (`-`), or equal (`=`)."""
    balls = range(1, N + 1)
    if isinstance(odd, str):
        sign = {'+': +1, '-': -1, '=': 0}
        oddballs = {sign[ch] * b for b in balls for ch in odd}
    else:
        oddballs = set(odd)
    return Puzzle(N, weighings, balls, oddballs)

# The `Weighing` class

`Weighing` is a simple data class:

In [4]:
@dataclass
class Weighing:
    """A weighing of the balls on the left (`L`) against the balls on the right (`R`),
    with three branches `gt`, `eq`, and `lt` for the three outcomes of the weighing."""
    L:  List[Ball]
    R:  List[Ball]
    gt: Tree
    eq: Tree
    lt: Tree

The function call `weigh(L, R, oddballs)` weighs the  `L` balls against the `R` balls assuming that only the given `oddballs` are possible. The three outcome branches partition the set of `oddballs`.

In [48]:
def weigh(L: Collection[Ball], R: Collection[Ball], oddballs: Set[Oddball]) -> Weighing:
    assert len(L) == len(R)
    weighing = Weighing(L, R, set(), set(), set())
    # Partition the oddballs into the three outcome branches
    for b in oddballs:
        if   +b in L or -b in R: weighing.gt.add(b)
        elif +b in R or -b in L: weighing.lt.add(b)
        else:                    weighing.eq.add(b)
    return weighing

Here's an example weighing where there are 5 balls, and the oddball can be lighter, heavier, or there might be no oddball:

In [49]:
weigh([1, 2], [4, 5], range(-5, 6))

Weighing(L=[1, 2], R=[4, 5], gt={1, 2, -5, -4}, eq={0, 3, -3}, lt={4, 5, -2, -1})

The notation `gt={1, 2, -5, -4}` means that if the outcome is that the left balance pan is heavier than the right then the remaining possibilities are that the 1 or 2 ball might be heavy, or the 4 or 5 ball might be light. 

# `Tree` Functions

Here are some straightforward utility functions on trees:

In [7]:
def is_solution(tree, puzzle) -> bool:
    """Is this policy tree a solution to the puzzle?
    It is if it all the leaves are singletons or empty, and every oddball appears."""
    return (all(len(leaf) <= 1 for leaf in leaves(tree)) 
            and set().union(*leaves(tree)) == puzzle.oddballs)

def leaves(tree: Weighing) -> List[Set[Oddball]]:
    """A list of all leaves of the tree."""
    return (leaves(tree.gt) + leaves(tree.eq) + leaves(tree.lt)
            if isinstance(tree, Weighing)
            else [tree])
    
def branches(tree: Weighing) -> tuple:
    """The three branches of a weighing."""
    return (tree.gt, tree.eq, tree.lt)

def is_good_partition(weighing: Weighing, w: int) -> bool:
    "Does this weighing partition the oddballs in a way that might be solved in `w` more weighings?"
    return all(len(oddballs) <= 3 ** w for oddballs in branches(weighing))

Let's write some simple tests for these functions:

In [125]:
def test1() -> bool:
    puzzle3 = make_puzzle(3, 1, odd='+')
    solution3 = weigh([1], [3], {1, 2, 3})
    
    assert solution3 == Weighing(L=[1], R=[3], gt={1}, eq={2}, lt={3})
    assert is_solution(solution3, puzzle3)
    assert leaves(solution3) == [{1}, {2}, {3}]
    assert branches(solution3) == ({1}, {2}, {3})

    assert make_puzzle(3, 1, '+').oddballs   == {1, 2, 3}
    assert make_puzzle(3, 1, '-').oddballs   == {-1, -2, -3}
    assert make_puzzle(3, 1, '+-').oddballs  == {1, 2, 3, -1, -2, -3}
    assert make_puzzle(3, 1, '+=').oddballs  == {0, 1, 2, 3}
    assert make_puzzle(3, 1, '+-=').oddballs == {0, 1, 2, 3, -1, -2, -3}
    assert make_puzzle(3, 1, '+-=') == Puzzle(N=3, weighings=1, balls=range(1, 4), oddballs={0, 1, 2, 3, -2, -3, -1})

    assert not is_good_partition(weigh([1, 2, 3],       [10, 11, 12], puzzle12.oddballs), 2)
    assert     is_good_partition(weigh([1, 2, 3, 4], [9, 10, 11, 12], puzzle12.oddballs), 2)
    
    return True

test1()

True

# Good and Bad Partitions

Consider a puzzle with 12 balls and 3 weighings. Assume we start by weighing balls [7, 8, 9] against [10, 11, 12]: 

In [97]:
puzzle12 = make_puzzle(N=12, weighings=3)

weigh([7, 8, 9], [10, 11, 12], puzzle12.oddballs)

Weighing(L=[7, 8, 9], R=[10, 11, 12], gt={7, 8, 9, -12, -11, -10}, eq={1, 2, 3, 4, 5, 6, -2, -6, -5, -4, -3, -1}, lt={10, 11, 12, -9, -8, -7})

If this was the first weighing in our policy tree, could we go on to solve the puzzle? 

The answer is: **No!** If the outcome is `eq` (the scale balances) then there are 12 possible oddballs remaining (any of 1–6 could be either heavy or light). We only have *w* = 2 weighings left, and 12 > 3<sup>2</sup>, so the **rule of 3** tells us it is impossible to reach a solution from here.  This a **bad partition**. 

Here's a **good partition**, in which no branch has more than 3<sup><i>2</i></sup> = 9 oddballs:

In [10]:
weigh([5, 6, 7, 8], [9, 10, 11, 12], puzzle12.oddballs)

Weighing(L=[5, 6, 7, 8], R=[9, 10, 11, 12], gt={5, 6, 7, 8, -12, -11, -10, -9}, eq={1, 2, 3, 4, -2, -4, -3, -1}, lt={9, 10, 11, 12, -8, -7, -6, -5})

# Solving Puzzles

So now we have a viable approach to implementing `solve(puzzle)`: build a policy tree that partitions oddballs down to singletons (or give up and return `None` if that can't be done in the allowable number of weighings). For each subset of oddballs in a partition, recursively call `solve` on a new subpuzzle. Since we don't know for sure which balls to weigh at each point, call `good_partitions` to generate multiple possible good partitions, and try to find one that works.

In [71]:
def solve(puzzle) -> Optional[Tree]:
    """Find a tree that covers all the oddballs in the puzzle within the given number of weighings."""
    N, w, oddballs = puzzle.N, puzzle.weighings, puzzle.oddballs
    if len(oddballs) <= 1:         # No choices left; this branch is complete
        return oddballs
    elif len(oddballs) > 3 ** w:   # Impossible to solve from here
        return None
    elif w > 0:                    # Find a partition that works
        for partition in good_partitions(puzzle):
            partition.lt = solve(make_puzzle(N, w - 1, partition.lt))
            partition.eq = solve(make_puzzle(N, w - 1, partition.eq))
            partition.gt = solve(make_puzzle(N, w - 1, partition.gt))
            if None not in branches(partition):
                return partition
    return None            # No solution; failure

Much of the work is done by `good_partitions`, a generator that yields good partitions. For now, I'm not going to try to be clever about choosing partitions. For values of `B` from 1 to `N/2`, I'll try to weigh the first `B`  balls on the left against the last `B` balls on the right. If that results ina good partition, I'll yield it. Then I'll randomly shuffle the balls and repeat the whole process (`repeat` times), so that I'll get different sets of balls on the left and right on each try.

In [101]:
def good_partitions(puzzle, repeat=100) -> Iterable[Weighing]:
    "Yield random good partitions."
    oddballs, w, balls = puzzle.oddballs, puzzle.weighings, sorted(puzzle.balls)
    for _ in range(repeat): 
        for B in range(1, len(balls) // 2 + 1):
            L, R = balls[:B], balls[-B:]
            partition = weigh(L, R, oddballs)
            if is_good_partition(partition, w - 1):
                yield partition
        random.shuffle(balls)  

We can check to see if `solve` does the job:

In [103]:
solve(puzzle12)

Weighing(L=[1, 2, 3, 4], R=[9, 10, 11, 12], gt=Weighing(L=[1, 8, 3, 7, 10], R=[2, 9, 5, 4, 6], gt=Weighing(L=[7, 4, 3], R=[11, 5, 1], gt={3}, eq={-9}, lt={1}), eq=Weighing(L=[1], R=[12], gt={-12}, eq={-11}, lt=set()), lt=Weighing(L=[4, 8, 9], R=[2, 11, 7], gt={4}, eq={-10}, lt={2})), eq=Weighing(L=[2, 1, 10, 9, 11], R=[6, 12, 5, 3, 8], gt=Weighing(L=[1, 2, 3, 4, 5], R=[8, 9, 10, 11, 12], gt={-8}, eq={-6}, lt={-5}), eq=Weighing(L=[1, 2, 3, 4, 5, 6], R=[7, 8, 9, 10, 11, 12], gt={-7}, eq=set(), lt={7}), lt=Weighing(L=[1, 2, 3, 4, 5], R=[8, 9, 10, 11, 12], gt={5}, eq={6}, lt={8})), lt=Weighing(L=[11, 4, 2, 10], R=[3, 12, 7, 6], gt=Weighing(L=[10, 5, 6, 12], R=[11, 8, 1, 9], gt={10}, eq={-3}, lt={11}), eq=Weighing(L=[1], R=[12], gt=set(), eq={9}, lt={-1}), lt=Weighing(L=[7, 11, 8, 10, 1], R=[12, 6, 4, 5, 3], gt={-4}, eq={-2}, lt={12})))

The good news is that it appears to get an answer; the bad news is that the output is really hard to read. 

Let's look at a trivial puzzle: 3 balls, 1 weighing allowed, and the oddball can only be heavier:

In [104]:
solve(make_puzzle(3, 1, odd='+'))

Weighing(L=[1], R=[3], gt={1}, eq={2}, lt={3})

This trivial policy tree says you weigh the first ball against the third (leaving the second unweighed), and the three possible weighing outcomes tell you which of the three balls  is heavier. That looks right.

# Prettier Output

Let's make the output easier to read. The function`do(puzzle)` will solve the puzzle, verify that the tree is valid, and print the tree in a pretty format using the function `pretty_tree(tree)`. The format is:
- Balls are represented by Unicode characters: `①` or `②`.
- An oddball is represented with a plus or minus sign for heavy or light: `+①` or `-②`.
- A weighing is represented by a string with a balance scale symbol: `[①② ⚖ ③④] ➔`.
- The three branches of a tree are represented by the prefixes "`>:`" or "`=:`" or "`<:`".
- The empty set of oddballs (meaning no ball is heavier) is represented by `∅`.
- An impossible branch is represented by the character `!` (as when weighing have ① > ② > ③).
- Branches are on indented lines, unless the branches are all leaves.

In [132]:
def do(puzzle) -> None:
    "Solve the puzzle; verify the solution; and print the solution in a pretty format."
    tree = solve(puzzle)
    assert (tree is None) or is_solution(tree, puzzle)
    print(pretty_tree(tree))
    
def pretty_tree(tree, depth=0, prefix='') -> str:
    "Pretty, indented string representing a policy tree."
    if tree is None:
        return 'No solution'
    elif isinstance(tree, Weighing):
        subtrees = (pretty_tree(tree.gt, depth + 1, '>:'), 
                    pretty_tree(tree.eq, depth + 1, '=:'), 
                    pretty_tree(tree.lt, depth + 1, '<:'))
        indent   = ('' if depth == 0 else ('\n' + '   ' * depth))
        return f'{indent}{prefix}[{ball_str(tree.L)} ⚖ {ball_str(tree.R)}] ➔ {" ".join(subtrees)}'
    else: # tree is a set of oddballs
        return f'{prefix}{oddball_str(tree)}'
    
def ball_str(balls: List[Ball]) -> str:
    """Unicode character string representing a list of balls."""
    return cat(BALLS[i] if (0 <= i < len(BALLS)) else f'({i})' 
               for i in sorted(balls))

def oddball_str(oddballs) -> str:
    """Unicode character string representing a list of oddballs.
    Normally a string like '-①+②', but the empty list returns '!'."""
    if not oddballs:
        return '!'
    else:
        return cat(("+" if b > 0 else "-" if b < 0 else "") + ball_str([abs(b)]) 
                   for b in oddballs)

cat = ''.join

BALLS = ('∅①②③④⑤⑥⑦⑧⑨⑩⑪⑫⑬⑭⑮⑯⑰⑱⑲⑳㉑㉒㉓㉔㉕㉖㉗㉘㉙㉚㉛㉜㉝㉞㉟㊱㊲㊳㊴㊵㊶㊷㊸㊹㊺㊻㊼㊽㊾㊿'
         'ⒶⒷⒸⒹⒺⒻⒼⒽⒾⒿⓀⓁⓂⓃⓄⓅⓆⓇⓈⓉⓊⓋⓌⓍⓎⓏ'
         'ⓐⓑⓒⓓⓔⓕⓖⓗⓘⓙⓚⓛⓜⓝⓞⓟⓠⓡⓢⓣⓤⓥⓦⓧⓨⓩ'
         '⦰⦱⦲⦳⦴⦵⦶⦷⦸⦹⦺⦻⦼⦽⦾⦿⧀⧁⧂⧃⨀⨁⨂⨴⨵⨶⨷⨸⓵⓶⓷⓸⓹⓺⓻⓼⓽⓾')

# Easy Puzzle Solutions

Now that the results will be readable, let's solve some puzzles:

In [133]:
# The trivial puzzle from before: 3 balls, 1 weighing, only heavier balls possible
do(make_puzzle(3, 1, odd='+'))

[① ⚖ ③] ➔ >:+① =:+② <:+③


In [134]:
# 9 balls, 2 weighings, odd ball can only be heavier
do(make_puzzle(9, 2, '+'))

[①②③ ⚖ ⑦⑧⑨] ➔ 
   >:[②⑤ ⚖ ③④] ➔ >:+② =:+① <:+③ 
   =:[①②③④ ⚖ ⑥⑦⑧⑨] ➔ >:+④ =:+⑤ <:+⑥ 
   <:[①④⑤⑧ ⚖ ②③⑥⑨] ➔ >:+⑧ =:+⑦ <:+⑨


In [130]:
# 3 balls, 2 weighings, lighter, heavier or equal balls possible
do(make_puzzle(3, 2, '+-='))

(① ⚖ ③) ➔ 
   >:(② ⚖ ①) ➔ >:! =:-③ <:+① 
   =:(② ⚖ ③) ➔ >:+② =:∅ <:-② 
   <:(① ⚖ ②) ➔ >:! =:+③ <:-①


Six of the outcomes (of the second weighings) correctly identify the numbered oddball. But there are three special outcomes:
- `=:∅` means there is no oddball; all balls weigh the same.
- `>:!` means this outcome is impossible (because it implies that `② > ① > ③`, and we know only one ball can have an odd weight).
- `<:!` also indicates an impossible outcome (because it implies `① < ③ < ②`).

Let's look at the original puzzle with 12 balls and 3 weighings. We'll run it twice and get two random solutions:

In [135]:
do(puzzle12)

[①②③④ ⚖ ⑨⑩⑪⑫] ➔ 
   >:[①②⑦⑫ ⚖ ③④⑥⑩] ➔ 
      >:[②⑥⑦⑪⑫ ⚖ ①③⑤⑧⑨] ➔ >:+② =:-⑩ <:+① 
      =:[①② ⚖ ⑪⑫] ➔ >:-⑪ =:-⑨ <:! 
      <:[④⑧⑩⑪⑫ ⚖ ①②⑤⑥⑦] ➔ >:+④ =:+③ <:-⑫ 
   =:[④⑧⑩⑫ ⚖ ②⑥⑦⑨] ➔ 
      >:[⑥⑧⑨⑩ ⚖ ①②③④] ➔ >:+⑧ =:-⑦ <:-⑥ 
      =:[①②③④⑤ ⚖ ⑧⑨⑩⑪⑫] ➔ >:+⑤ =:! <:-⑤ 
      <:[⑥⑧⑪ ⚖ ①②④] ➔ >:+⑥ =:+⑦ <:-⑧ 
   <:[①③⑦⑩ ⚖ ④⑤⑧⑨] ➔ 
      >:[①②③ ⚖ ⑩⑪⑫] ➔ >:! =:-④ <:+⑩ 
      =:[⑫ ⚖ ⑪] ➔ >:+⑫ =:-② <:+⑪ 
      <:[⑤⑦⑧ ⚖ ③⑨⑩] ➔ >:-③ =:-① <:+⑨


In [136]:
do(puzzle12)

[①②③④ ⚖ ⑨⑩⑪⑫] ➔ 
   >:[③⑦⑫ ⚖ ②⑨⑩] ➔ 
      >:[②⑩ ⚖ ⑦⑨] ➔ >:-⑨ =:+③ <:-⑩ 
      =:[①②⑤ ⚖ ④⑥⑩] ➔ >:+① =:-⑪ <:+④ 
      <:[① ⚖ ⑫] ➔ >:-⑫ =:+② <:! 
   =:[②③④⑦⑧ ⚖ ①⑥⑨⑪⑫] ➔ 
      >:[②⑤⑦⑨⑩ ⚖ ③④⑧⑪⑫] ➔ >:+⑦ =:-⑥ <:+⑧ 
      =:[①②③④⑤ ⚖ ⑧⑨⑩⑪⑫] ➔ >:+⑤ =:! <:-⑤ 
      <:[①③⑥⑦⑨ ⚖ ②④⑤⑩⑪] ➔ >:+⑥ =:-⑧ <:-⑦ 
   <:[④⑤⑨ ⚖ ①②⑪] ➔ 
      >:[②⑪ ⚖ ①⑤] ➔ >:-① =:+⑨ <:-② 
      =:[③⑤⑥⑧⑩ ⚖ ②④⑦⑨⑪] ➔ >:+⑩ =:+⑫ <:-③ 
      <:[①② ⚖ ⑪⑫] ➔ >:! =:-④ <:+⑪


I note that the traditional answer to the 12-ball puzzle weighs 4 balls on each side of the first weighing, three balls on the second weighing, and 1 ball on each side on the third weighing. But my program  won't necessarily minimize the number of balls on each side of the scale, nor make one branch of the tree be symmetric with another branch.

# Other Puzzles with 3 Weighings



We can do **12 balls in 3 weighings** even when we add in the possibility that there is no oddball–that all the balls weigh the same:

In [24]:
do(make_puzzle(12, 3, odd='+-='))

①②③④ ⚖ ⑨⑩⑪⑫ ➔ 
   >:④⑧⑫ ⚖ ③⑩⑪ ➔ 
      >:④⑤⑥⑩ ⚖ ①③⑦⑧ ➔ >:+④ =:-⑪ <:-⑩ 
      =:②④ ⚖ ①⑩ ➔ >:+② =:-⑨ <:+① 
      <:① ⚖ ⑫ ➔ >:-⑫ =:+③ <:! 
   =:①②③⑨ ⚖ ⑥⑦⑧⑪ ➔ 
      >:③⑦ ⚖ ⑧⑪ ➔ >:-⑧ =:-⑥ <:-⑦ 
      =:①②③④⑤ ⚖ ⑧⑨⑩⑪⑫ ➔ >:+⑤ =:∅ <:-⑤ 
      <:①⑦ ⚖ ②⑥ ➔ >:+⑦ =:+⑧ <:+⑥ 
   <:②⑤⑦⑧ ⚖ ①③④⑩ ➔ 
      >:①②⑧⑩⑫ ⚖ ③⑤⑦⑨⑪ ➔ >:-③ =:-④ <:-① 
      =:②⑪ ⚖ ⑧⑫ ➔ >:+⑪ =:+⑨ <:+⑫ 
      <:①② ⚖ ⑪⑫ ➔ >:! =:+⑩ <:-②


Can we solve the **13-light-or-heavy-balls in 3 weighings** puzzle, which has 26 possibilities? After all, 26 is less than 3<sup>3</sup> = 27.

In [25]:
puzzle13 = make_puzzle(13, 3, odd='+-')
do(puzzle13)

No solution


**No**, and we can see why by looking at the sizes of the three partitions. For the first weighing we need to find some number of balls *B* to put on each side of the scale so that there will be no more than 9 balls in any partition. It seems best to choose a *B* that is near *N*/3, where *N* is the number of balls in the puzzle, to partition the oddballs into thirds. 

So for *N* = 13, it looks like *B* = 4 or 5 balls is the best bet, but to help figure it out, I'll define the function `partition_sizes` to build a dict that, for various numbers *B* of balls gives the sizes of the three resulting branches of the partition, and tells whether the partition is *good* or *bad*.

In [26]:
def partition_sizes(puzzle) -> Dict[int, Tuple[int, int, int]]:
    """For various nunbers of balls B to weigh, what are the sizes of the three branches,
    and is the partition good?"""
    third = puzzle.N // 3
    result = {}
    for B in range(third - 2, third + 4):
        L, R = puzzle.balls[:B], puzzle.balls[-B:]
        partition = weigh(L, R, puzzle.oddballs)
        lengths = map(len, branches(partition))
        good = 'Good' if is_good_partition(partition, puzzle.weighings - 1) else 'Bad'
        result[B] = (good, *lengths)
    return result   

In [27]:
partition_sizes(puzzle13)

{2: ('Bad', 4, 18, 4),
 3: ('Bad', 6, 14, 6),
 4: ('Bad', 8, 10, 8),
 5: ('Bad', 10, 6, 10),
 6: ('Bad', 12, 2, 12),
 7: ('Bad', 14, 0, 12)}

We see that whatever value we choose for `B`, we will always get a bad partition of the 13-ball puzzle (there will always be at least one branch with more than 9 oddballs). 

For the puzzle with 12 balls (and thus 24 oddballs), the only good partition is with 4 balls on each side of the balance:

In [28]:
partition_sizes(puzzle12)

{2: ('Bad', 4, 16, 4),
 3: ('Bad', 6, 12, 6),
 4: ('Good', 8, 8, 8),
 5: ('Bad', 10, 4, 10),
 6: ('Bad', 12, 0, 12),
 7: ('Bad', 14, 0, 10)}

We can do **27 balls in 3 weighings** if we know that the oddball can only be heavier, not lighter:

In [29]:
puzzle27 = make_puzzle(27, 3, '+')
partition_sizes(puzzle27)

{7: ('Bad', 7, 13, 7),
 8: ('Bad', 8, 11, 8),
 9: ('Good', 9, 9, 9),
 10: ('Bad', 10, 7, 10),
 11: ('Bad', 11, 5, 11),
 12: ('Bad', 12, 3, 12)}

In [30]:
do(puzzle27)

①②③④⑤⑥⑦⑧⑨ ⚖ ⑲⑳㉑㉒㉓㉔㉕㉖㉗ ➔ 
   >:④⑥⑧⑬⑭⑱⑲㉒㉕ ⚖ ①②⑨⑩⑮⑰⑳㉓㉔ ➔ 
      >:④⑳ ⚖ ⑧⑫ ➔ >:+④ =:+⑥ <:+⑧ 
      =:④⑤⑬⑭⑱㉖ ⚖ ⑦⑨⑩⑪⑯⑲ ➔ >:+⑤ =:+③ <:+⑦ 
      <:④⑨⑭⑮⑯⑱⑳㉔㉕㉗ ⚖ ①③⑤⑥⑦⑧⑩⑬⑲㉓ ➔ >:+⑨ =:+② <:+① 
   =:①②③④⑤⑥⑦⑧⑨⑩⑪⑫ ⚖ ⑯⑰⑱⑲⑳㉑㉒㉓㉔㉕㉖㉗ ➔ 
      >:④⑤⑦⑩⑭⑮⑳ ⚖ ①⑧⑨⑫⑰㉒㉖ ➔ >:+⑩ =:+⑪ <:+⑫ 
      =:①②③④⑤⑥⑦⑧⑨⑩⑪⑫⑬ ⚖ ⑮⑯⑰⑱⑲⑳㉑㉒㉓㉔㉕㉖㉗ ➔ >:+⑬ =:+⑭ <:+⑮ 
      <:②④⑩⑪⑭⑮⑯⑲㉒㉔㉕ ⚖ ③⑤⑦⑧⑬⑰⑳㉑㉓㉖㉗ ➔ >:+⑯ =:+⑱ <:+⑰ 
   <:②⑦⑨⑮⑯㉒㉓㉔ ⚖ ③⑥⑩⑬⑱⑳㉑㉕ ➔ 
      >:①⑤⑥⑪⑱⑲㉑㉔㉖ ⚖ ④⑦⑨⑩⑫⑭⑰⑳㉒ ➔ >:+㉔ =:+㉓ <:+㉒ 
      =:⑤⑯㉓㉗ ⚖ ①⑪⑱㉖ ➔ >:+㉗ =:+⑲ <:+㉖ 
      <:①③④⑥㉑ ⚖ ⑨⑫⑬㉔㉕ ➔ >:+㉑ =:+⑳ <:+㉕


And we can do **26 balls** under the condition that either one ball is heavier or all the balls weigh the same (so, 27 possibilities):

In [137]:
do(make_puzzle(26, 3, '+='))

[①②③④⑤⑥⑦⑧⑨ ⚖ ⑱⑲⑳㉑㉒㉓㉔㉕㉖] ➔ 
   >:[②③⑤⑮⑱㉑ ⚖ ①⑥⑨⑫㉔㉕] ➔ 
      >:[②⑦㉓ ⚖ ③㉔㉖] ➔ >:+② =:+⑤ <:+③ 
      =:[③⑧⑩⑬⑮⑰⑱⑳㉓㉔㉕㉖ ⚖ ①②⑤⑥⑦⑨⑪⑫⑭⑯⑲㉑] ➔ >:+⑧ =:+④ <:+⑦ 
      <:[①⑧⑩⑱⑲㉓㉖ ⚖ ③④⑦⑨⑰㉔㉕] ➔ >:+① =:+⑥ <:+⑨ 
   =:[①②③④⑤⑥⑦⑧⑨⑩⑪⑫ ⚖ ⑮⑯⑰⑱⑲⑳㉑㉒㉓㉔㉕㉖] ➔ 
      >:[④⑪ ⚖ ⑩⑮] ➔ >:+⑪ =:+⑫ <:+⑩ 
      =:[①②③④⑤⑥⑦⑧⑨⑩⑪⑫⑬ ⚖ ⑭⑮⑯⑰⑱⑲⑳㉑㉒㉓㉔㉕㉖] ➔ >:+⑬ =:∅ <:+⑭ 
      <:[③⑦⑧⑭⑯⑱㉒㉓㉔㉖ ⚖ ①②④⑤⑩⑫⑬⑰⑲⑳] ➔ >:+⑯ =:+⑮ <:+⑰ 
   <:[③⑩⑮㉓㉔㉕ ⚖ ②⑦⑪⑱㉑㉒] ➔ 
      >:[⑫⑬⑮⑯㉓ ⚖ ④⑥⑲㉕㉖] ➔ >:+㉓ =:+㉔ <:+㉕ 
      =:[②⑦⑧⑨⑫⑯⑱⑲㉑㉓㉔ ⚖ ①③⑤⑥⑩⑪⑭⑮⑰㉕㉖] ➔ >:+⑲ =:+⑳ <:+㉖ 
      <:[③④⑧⑨⑪⑭⑲⑳㉒㉓㉕ ⚖ ①②⑤⑥⑩⑫⑬⑮⑰⑱㉖] ➔ >:+㉒ =:+㉑ <:+⑱


Here's another puzzle with **26 balls**, where it is stipulated that either one of the odd-numbered balls is heavier, or one of the even-numbered balls is lighter, or there is no oddball.

In [32]:
odd_even = {(+b if b % 2 else -b) for b in range(27)}
print(odd_even)

{0, 1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, -26, -24, -22, -20, -18, -16, -14, -12, -10, -8, -6, -4, -2}


In [51]:
do(make_puzzle(26, 3, odd=odd_even))

①③⑫⑭⑮⑯⑰㉒㉔ ⚖ ②⑤⑧⑩⑬⑱㉑㉕㉖ ➔ 
   >:①②⑨⑩⑳㉓㉕ ⚖ ⑤⑥⑦⑧⑪⑰㉖ ➔ 
      >:②③⑧⑨⑮⑳㉕ ⚖ ⑤⑦⑪⑲㉑㉔㉖ ➔ >:-㉖ =:+① <:-⑧ 
      =:⑥⑫⑮⑱㉑㉓㉕ ⚖ ④⑤⑬⑰⑳㉔㉖ ➔ >:+⑮ =:+③ <:-⑱ 
      <:⑨⑬⑲㉑㉓ ⚖ ②⑤⑯⑰⑳ ➔ >:-② =:-⑩ <:+⑰ 
   =:⑤⑥⑦⑧⑪⑱㉑㉕ ⚖ ③⑨⑩⑫⑮⑲⑳㉔ ➔ 
      >:②⑦⑭⑳㉑ ⚖ ⑫⑮⑯⑱㉕ ➔ >:+⑦ =:+⑪ <:-⑳ 
      =:⑧⑩⑰⑱㉒ ⚖ ④⑦⑯㉓㉕ ➔ >:-④ =:∅ <:+㉓ 
      <:③⑦⑩⑭⑮⑱⑳㉑㉓㉔㉖ ⚖ ①②④⑤⑥⑪⑫⑬⑯⑰⑲ ➔ >:-⑥ =:+⑨ <:+⑲ 
   <:⑤⑥⑯⑲⑳㉕㉖ ⚖ ②④⑧⑨⑬㉑㉒ ➔ 
      >:⑦⑧⑨⑬⑭⑳㉔㉕ ⚖ ①③⑤⑫⑮⑯⑰⑱ ➔ >:+㉕ =:-㉒ <:+⑤ 
      =:①②③④⑤⑥⑦⑧⑨⑩⑪⑫ ⚖ ⑮⑯⑰⑱⑲⑳㉑㉒㉓㉔㉕㉖ ➔ >:-㉔ =:-⑭ <:-⑫ 
      <:①②③④⑤⑥⑦⑧⑨⑩⑪ ⚖ ⑯⑰⑱⑲⑳㉑㉒㉓㉔㉕㉖ ➔ >:-⑯ =:+⑬ <:+㉑


Another variation: **27 balls**, 3 weighings, the oddball can only be heavier, but one ball, number 27, is toxic and can't be touched or placed on the balance scale. It might, however be the heavier ball, and you still need to report it as such if it is. 

We describe this situation by defining a puzzle with 26 (weighable) balls, but with the oddballs including the positive ball numbers from +1 to +27, inclusive. Note  the correct `=:+㉗` notation when all three weighings are equal.

In [34]:
do(make_puzzle(26, 3, odd=range(1, 28)))

①②③④⑤⑥⑦⑧⑨ ⚖ ⑱⑲⑳㉑㉒㉓㉔㉕㉖ ➔ 
   >:①②③⑩⑱⑳㉓ ⚖ ④⑤⑥⑯⑲㉔㉕ ➔ 
      >:①⑭⑳ ⚖ ③⑦㉖ ➔ >:+① =:+② <:+③ 
      =:⑨⑮㉑ ⚖ ①⑦⑫ ➔ >:+⑨ =:+⑧ <:+⑦ 
      <:④⑪⑫⑭⑮⑱ ⚖ ⑥⑦⑲⑳㉑㉒ ➔ >:+④ =:+⑤ <:+⑥ 
   =:①②③④⑤⑥⑦⑧⑨⑩⑪⑫ ⚖ ⑮⑯⑰⑱⑲⑳㉑㉒㉓㉔㉕㉖ ➔ 
      >:①②⑤⑥⑧⑨⑩⑬⑮⑰⑲⑳ ⚖ ③④⑦⑫⑭⑱㉑㉒㉓㉔㉕㉖ ➔ >:+⑩ =:+⑪ <:+⑫ 
      =:①②③④⑤⑥⑦⑧⑨⑩⑪⑫⑬ ⚖ ⑭⑮⑯⑰⑱⑲⑳㉑㉒㉓㉔㉕㉖ ➔ >:+⑬ =:+㉗ <:+⑭ 
      <:②③⑦⑭⑯㉑㉒㉔ ⚖ ①④⑨⑩⑬⑰⑱㉖ ➔ >:+⑯ =:+⑮ <:+⑰ 
   <:②④⑤⑥⑫⑱⑳㉔ ⚖ ①⑦⑨⑭⑰⑲㉒㉓ ➔ 
      >:①④⑦⑨⑭⑯⑰㉑㉓㉔㉕ ⚖ ③⑥⑧⑩⑪⑫⑮⑲⑳㉒㉖ ➔ >:+㉔ =:+⑱ <:+⑳ 
      =:③⑤⑥⑨⑩⑭⑱⑲㉖ ⚖ ①⑦⑧⑪⑫⑬⑰㉒㉕ ➔ >:+㉖ =:+㉑ <:+㉕ 
      <:③⑤⑥⑧⑪⑯㉒㉔ ⚖ ①②⑦⑮⑳㉑㉓㉖ ➔ >:+㉒ =:+⑲ <:+㉓


# Puzzles with 4 Weighings

We can tackle larger puzzles. With 4 weighings, we can theoretically handle up to 3<sup>4</sup> = 81 possibilities. Can we solve for **40 balls**, heavy or light? Let's check the partition sizes:

In [35]:
partition_sizes(make_puzzle(40, 4))

{11: ('Bad', 22, 36, 22),
 12: ('Bad', 24, 32, 24),
 13: ('Bad', 26, 28, 26),
 14: ('Bad', 28, 24, 28),
 15: ('Bad', 30, 20, 30),
 16: ('Bad', 32, 16, 32)}

Every partition is bad, leaving us with a branch of size 28 or more, which can't be handled in three remaining weighings.

How about **39 balls**, heavier or lighter, with the possibility that no ball is odd? 

In [36]:
puzzle39 = make_puzzle(39, 4, '+-=')

In [37]:
partition_sizes(puzzle39)

{11: ('Bad', 22, 35, 22),
 12: ('Bad', 24, 31, 24),
 13: ('Good', 26, 27, 26),
 14: ('Bad', 28, 23, 28),
 15: ('Bad', 30, 19, 30),
 16: ('Bad', 32, 15, 32)}

We see that it might be possible to solve the puzzle by starting with 13 balls on each side. Let's try:

In [38]:
do(puzzle39)

①②③④⑤⑥⑦⑧⑨⑩⑪⑫⑬ ⚖ ㉗㉘㉙㉚㉛㉜㉝㉞㉟㊱㊲㊳㊴ ➔ 
   >:④⑤⑧⑨⑭⑰㉑㉕㉗㊱㊳ ⚖ ②③⑥⑦⑪⑬㉒㉚㉜㉞㉟ ➔ 
      >:①③⑦⑨⑫⑬⑭⑯㉑㉕㉘㉚㉟㊴ ⚖ ④⑥⑩⑮⑰⑲⑳㉓㉔㉙㉛㉜㉝㊳ ➔ 
         >:①②③④⑤⑥⑦⑧ ⚖ ㉜㉝㉞㉟㊱㊲㊳㊴ ➔ >:-㉜ =:+⑨ <:! 
         =:②④⑥⑧⑫⑱㉓㉔㊴ ⚖ ⑤⑪⑭⑲㉗㉘㉙㊲㊳ ➔ >:+⑧ =:-㉞ <:+⑤ 
         <:③④⑫⑬⑮⑯㉔㉕㉝㉟㊴ ⚖ ②⑥⑲⑳㉑㉒㉓㉖㉛㉞㊱ ➔ >:+④ =:-㉚ <:-㉟ 
      =:②③⑦⑩⑪⑭⑮⑰⑱㉖㉝㉞㊱㊴ ⚖ ①④⑤⑥⑬⑲⑳㉑㉒㉓㉔㉕㉙㊲ ➔ 
         >:④⑤⑳㉒㉓㉖㉚㉝㉟㊱㊴ ⚖ ②⑨⑩⑫⑭⑯⑲㉗㉜㉞㊲ ➔ >:-㊲ =:-㉙ <:+⑩ 
         =:①⑤⑥⑨⑪⑭⑳㉔㉕㊳ ⚖ ③④⑫⑮⑱㉑㉓㉘㉝㊱ ➔ >:-㉘ =:-㉛ <:+⑫ 
         <:①⑤⑦⑧⑩⑯㉑㉓㉛㊳㊴ ⚖ ②③⑥⑭⑱⑲㉒㉔㉘㉚㉞ ➔ >:+① =:-㉝ <:-㊴ 
      <:③⑦⑧⑨⑬㉓㉖㉗㉙㉚㉜㉝㉟㊱ ⚖ ①④⑤⑥⑮⑯⑰⑱⑳㉒㉕㉛㉞㊲ ➔ 
         >:⑤⑩⑫⑬⑮㉜㉞ ⚖ ⑦⑰⑱㉒㉖㉚㊳ ➔ >:+⑬ =:+③ <:+⑦ 
         =:②㊴ ⚖ ⑪⑭ ➔ >:+② =:-㊳ <:+⑪ 
         <:②③⑦⑧⑩⑪⑬⑭⑯㉑㉒㉓㉕㉖㉙㉚㉜㉞ ⚖ ①④⑤⑥⑨⑰⑱⑲⑳㉔㉘㉛㉝㉟㊱㊲㊳㊴ ➔ >:-㊱ =:-㉗ <:+⑥ 
   =:⑨⑬⑭⑯⑰⑲㉒㉕㉚㉜㊴ ⚖ ⑥⑦⑧⑪⑫⑱㉑㉔㉗㉘㊳ ➔ 
      >:③⑤⑪⑬⑭⑮⑯㉑ ⚖ ②⑨⑰⑲㉔㉖㉙㉟ ➔ 
         >:②④⑧⑯㉖㉘ ⚖ ①⑥⑨⑩⑭㉟ ➔ >:+⑯ =:-㉔ <:+⑭ 
         =:⑤⑥⑦⑩⑭⑯㉒㉖㉗㉛㉝㊲㊳ ⚖ ②③⑧⑪⑫⑬⑮⑰⑳㉑㉔㉕㉟ ➔ >:+㉒ =:-⑱ <:+㉕ 
         <:⑧⑯㉔㉗㉟ ⚖ ⑬⑭⑲㉑㊲ ➔ >:-㉑ =:+⑰ <:+⑲ 
      =:①②③④⑤⑥⑦⑧⑨⑩⑪⑫⑬⑭⑮⑯⑰ ⚖ ㉓㉔㉕㉖㉗㉘㉙㉚㉛㉜㉝㉞㉟㊱㊲㊳㊴ ➔ 
         >:①④⑥⑨⑪⑫⑯⑲⑳ ⚖ ③⑦⑮㉖㉙㉞㉟㊱㊲ ➔ >:-㉖ =:-㉓ <:+⑮ 
         =:③④⑩⑪⑬⑮⑯⑰⑱⑲⑳

How about **80 balls, 4 weighings** under the condition that no ball can be heavier (thus, 81 possibilities, the maximum)?

In [138]:
do(make_puzzle(80, 4, '-='))

[①②③④⑤⑥⑦⑧⑨⑩⑪⑫⑬⑭⑮⑯⑰⑱⑲⑳㉑㉒㉓㉔㉕㉖㉗ ⚖ ⒹⒺⒻⒼⒽⒾⒿⓀⓁⓂⓃⓄⓅⓆⓇⓈⓉⓊⓋⓌⓍⓎⓏⓐⓑⓒⓓ] ➔ 
   >:[①③⑦⑬㉒㉔㉗㉜㊱㊵㊷㊿ⒽⓁⓅⓇⓈⓉⓍⓑⓒ ⚖ ⑥⑨⑩⑲⑳㉕㉙㉚㊳㊶㊾ⒷⒹⒻⒼⒾⓂⓃⓆⓊⓐ] ➔ 
      >:[⑥⑧⑬㉝㊲㊿ⒶⒻⓁⓂⓃⓌⓏ ⚖ ③㉒㉔㉖㉘㊽ⒹⒼⒽⒿⓐⓒⓓ] ➔ 
         >:[⑤⑫⑲㉓㉛ⒷⒸⒽⒾⓐⓒ ⚖ ①②③④⑬㉕㊷ⒹⓂⓄⓋ] ➔ >:-Ⓓ =:-Ⓖ <:-ⓐ 
         =:[④⑥⑪⑯⑲㉓㉕㉗㉘㉙㉝㉟㊱㊵㊹㊺㊻㊿ⒶⒺⒻⒼⒽⓂⓃⓅⓆⓈⓌⓒ ⚖ ①⑤⑦⑬⑭⑮⑱㉚㉜㉞㊲㊴㊷㊸㊼㊽㊾ⒷⒸⒹⒾⒿⓀⓁⓋⓍⓎⓏⓑⓓ] ➔ >:-Ⓘ =:-Ⓤ <:-Ⓠ 
         <:[⑧⑨⑩㉚㊲ⓂⓅⓈ ⚖ ⑫㊱㊹ⒷⒸⒻⒼⒿ] ➔ >:-Ⓕ =:-Ⓝ <:-Ⓜ 
      =:[②⑲㉕㉘㉙㉟㊱㊴㊵㊽㊿ⒼⓀⓄⓆⓉⓎⓑ ⚖ ⑥⑧⑨⑫⑰⑱㉓㉔㉗㉞㊸ⒸⒹⒿⓂⓈⓌⓏ] ➔ 
         >:[①⑥⑦⑫⑱㉒㉔㉖㉚㊳㊷㊸㊻㊽ⒶⒷⒸⒼⓁⓂⓆⓇⓏⓓ ⚖ ④⑤⑧⑩⑬⑲㉗㉛㉝㉞㉟㊱㊴㊶㊹㊺㊾㊿ⒽⒾⓃⓌⓎⓑ] ➔ >:-Ⓦ =:-Ⓙ <:-Ⓩ 
         =:[③④⑥⑬⑰⑱⑲㉜㉝㉞㉟㊺㊻ⒼⒾⒿⓁⓆⓇⓏⓐⓓ ⚖ ①②⑤⑦⑧⑨⑭⑳㉓㉕㉗㊲㊳㊸㊾ⒶⒷⒸⓀⓂⓊⓋ] ➔ >:-Ⓥ =:-Ⓔ <:-ⓓ 
         <:[①④⑦⑩⑫⑬㉒㉖㉗㉘㉟㊱㊳㊴㊶㊹㊿ⒶⒷⒹⒼⒾⒿⓅⓇⓋⓌⓎⓐⓑ ⚖ ⑥⑧⑨⑪⑮⑯⑰⑱⑲㉑㉓㉔㉕㉚㉛㊵㊸㊻㊼㊽㊾ⒺⒽⓁⓃⓄⓆⓈⓉⓏ] ➔ >:-Ⓞ =:-Ⓚ <:-Ⓨ 
      <:[④⑦⑩⑪⑬⑱㉓㉔㉖㉗㉚㉜㊳㊴㊺ⒶⒸⒹⒾⓀⓃⓆⓇⓉⓌⓏⓑ ⚖ ①③⑥⑧⑨⑭⑲㉒㉛㉝㊵㊷㊸㊻㊽㊾㊿ⒺⒿⓁⓂⓄⓅⓈⓋⓎⓐ] ➔ 
         >:[④⑲㉕㉚㊳㊹ⒷⒾⓅⓌⓑⓒ ⚖ ②㉑㉓㉔㉝㉟㊺ⓁⓄⓆⓏⓓ] ➔ >:-Ⓛ =:-Ⓢ <:-Ⓟ 
         =:[②⑨⑬⑭㉚㊴㊵㊸㊺㊼㊾ⒷⒸⒹⒼⒾⒿⓃⓏⓒ ⚖ ④⑧⑩⑯⑱㉓㉔㉖㉘㉛㉜㉟㊳㊷ⒶⓊⓌⓍⓎⓐ] ➔ >:-Ⓧ =:-Ⓗ <:-ⓒ 
         <:[③㊲ⒹⓂⓐⓑ ⚖ ㊷ⒺⓀⓄⓇⓍ] ➔ >:-Ⓡ =:-Ⓣ <:-ⓑ 
   =:[①②③④⑤⑥⑦⑧⑨⑩⑪⑫⑬⑭⑮⑯⑰⑱⑲⑳㉑㉒㉓㉔㉕㉖㉗㉘㉙㉚㉛㉜㉝㉞㉟㊱ ⚖ ㊺㊻㊼㊽㊾㊿ⒶⒷⒸⒹⒺⒻⒼⒽⒾⒿⓀⓁⓂⓃⓄⓅⓆⓇⓈⓉⓊⓋ

Looking good (except that the output is a bit hard to read and understand, because it is so large).

# Puzzles with 5 or More Weighings

With 5 weighings I could conceivably handle up to 3<sup>5</sup> = 243 oddballs, and with 6 weighings, 3<sup>6</sup> = 729. The resulting policy trees would be multiple pages of output, so I don't think I need to see them. Instead, I'll make a **table** of results. For various values of *w* from 3 up, and for the oddballs being each of '+-', '+-=', '+', '+=', I want to know what's the highest number of balls, *N*, that leads to a solvable puzzle (and the total number of oddballs, which will be either *N*, 2*N*, *N* + 1, or 2*N* + 1).

In [139]:
def report(w_range=range(3, 7), odds=('+-', '+-=', '+', '+=')):
    headers = ['*w*','3<sup>*w*</sup>', *(f'odd: "{odd}"' for odd in odds)]
    return format_table(headers, [[w, 3 ** w, *(highest_N(w, odd) for odd in odds)] 
                                  for w in w_range])
    
def highest_N(w, odd: str) -> Optional[Tuple[int, int]]:
    """Find highest number of balls N (and total number of oddballs) that are solvable in w weighings."""
    target = 3 ** w // (odd.count('+') + odd.count('-'))
    for N in reversed(range(target + 3)):
        puzzle = make_puzzle(N, w, odd)
        if any(good_partitions(puzzle, repeat=1)) and solve(puzzle):
            return (N, len(puzzle.oddballs))
    return None # No solution

I decided to format the table with IPython Markdown:

In [140]:
from IPython.display import Markdown

def format_table(headers, rows, justify='---:') -> Markdown:
    """Make a markdown table from header and rows."""
    underline = [justify for _ in headers]
    lines = [headers, underline, *rows]
    return Markdown(cat(map(format_row, lines)))

def format_row(columns) -> str: 
    """Format a single row of a table in Markdown format."""
    columns = map(str, columns)
    return "|" + "|".join(columns) + "|\n"

In [58]:
%time report(range(1, 6))

CPU times: user 5.39 s, sys: 17.4 ms, total: 5.41 s
Wall time: 5.41 s


|*w*|3<sup>*w*</sup>|odd: "+-"|odd: "+-="|odd: "+"|odd: "+="|
|---:|---:|---:|---:|---:|---:|
|1|3|None|None|(3, 3)|(2, 3)|
|2|9|(3, 6)|(3, 7)|(9, 9)|(8, 9)|
|3|27|(12, 24)|(12, 25)|(27, 27)|(26, 27)|
|4|81|(39, 78)|(39, 79)|(81, 81)|(80, 81)|
|5|243|(120, 240)|(120, 241)|(243, 243)|(242, 243)|


# What's Next?

- What other puzzles can you solve?
- Can you *prove* the unsolved puzzles are unsolvable? 
- Can you find trees that minimize the *mean* number of weighings, rather than minimizing the *maximum* number of weighings?
- Can you minimize the number of balls in each weighing? The solutions above sometimes weigh 3 or 4 balls on each side of the scale when only 1 or 2 were necessary.
- Currently, when `solve` returns `None`, it means "no solution was found, but there's no proof that a solution doesn't exist." Can you modify `solve` to return a result that indicates there is no possible solution? (Maybe you can only do this in a reasonable amount of time on smaller puzzles, not all.)
- What about puzzles where your task is to identify *which* ball is odd, but not *how* it is odd? That is, if you get the possible oddballs down to `{-3, +3}` then you're done; you know ball 3 is the odd one, and you don't care if it is heavy or light.
- More generally, can you solve puzzles when it is a possibility (or a requirement) that *two* balls are odd?  Or more than two?
- What if you had two or more balance scales that you could use in parallel, and the goal was to minimize the number of weighing time periods, not the total number of weighings?
- What else can you discover?

# Better Partitions

I defined `good_partitions` to randomly guess at a good partition; that worked fine for small puzzles. But for larger puzzles, I want to do a more systematic job. Here's what I'm thinking:

- I should be focusing on **oddballs**, not balls.
- Balls with the same information should be treated the same. For example, at the start of a puzzle where all balls might be lighter or heavier, it matters how many balls I put on each side of the scale, but it doesn't matter which balls; `①②③④` is equivalent to `③⑥⑧⑪`.
- Balls with different information should be treated differently. If the set of oddballs has the subset {+1, -1, +2} then I get more information from putting ball 1 on the scale (it might turn out to be heavy or light) than ball 2 (which we already know can not be light).

In [152]:
def divide(puzzle) -> Dict[str, Set[Ball]]:
    oddballs, balls = puzzle.oddballs, puzzle.balls
    return {'+-': {b for b in balls if b in oddballs and -b in oddballs},
            '+':  {b for b in balls if b in oddballs and -b not in oddballs},
            '-':  {b for b in balls if -b in oddballs and b not in oddballs},
            '':   {b for b in balls if b not in oddballs and -b not in oddballs}}

puz = Puzzle(8, 3, balls=range(1, 9), oddballs={0, 1, -1, 2, -2, 3, -4, 5, -6})
divide(puz)

{'+-': {1, 2}, '+': {3, 5}, '-': {4, 6}, '': {7, 8}}

In [43]:
from collections import defaultdict

def divide(oddballs):
    dic = defaultdict(list)
    for ball in set(map(abs, oddballs)) - {0}:
        dic[ball in oddballs, -ball in oddballs].append(ball)
    return [(sum(x), dic[x]) for x in dic]

def pick_L(division, points_needed: int) -> Iterator[List[Ball]]:
    #print('call pick_L', division, points_needed)
    if points_needed == 0:
        yield []
    elif points_needed > 0 and division:
        [(points, balls), *rest] = division
        for p in range(0, points_needed // points + 1):
            #print('  working on', balls[:p])
            for remainder in pick_L(rest, points_needed - p * points):
                yield balls[:p] + remainder

def good_partitionsXXX(puzzle) -> Iterable[Weighing]:
    "Yield good partitions."
    oddballs, weighings, balls = puzzle.oddballs, puzzle.weighings, list(puzzle.balls)
    if isinstance(puzzle.odd, str):
        tries = 1 # First weighing, undifferentiated balls: only need to try once.
    for _ in range(tries): 
        for B in range(1, 1 + len(balls) // 2):
            L, R = balls[:B], balls[-B:]
            partition = weigh(L, R, oddballs)
            if is_good_partition(partition, weighings - 1):
                yield partition
        random.shuffle(balls)

list(pick_L(divide(puzzle12.oddballs), 6))

[[1, 2, 3]]

In [44]:
# sorted(range(1, N // 2 + 1), key=distance_to(N / 3)):
def distance_to(target): return lambda x: abs(x - target) 

